centroid connector builder
description: generate centroid connectors for OSM drive network
Sijia.Wang@wsp.com
Feb 7, 2019

In [239]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, shape, LineString
from scipy.spatial import cKDTree

In [306]:
#define input files
#osm drive network (edge/node), centroid connector file, node coordinates
cd = "C:/Users/wangs1/Documents/Met_Council/"
osm_edge_file = cd + "data/network-shape_MetCouncil_full/osm_networktype_drive/consolidate_lane/osm_complete_lanes.shp"
osm_node_file = cd + "data/network-shape_MetCouncil_full/osm_networktype_drive/drive_node.shp"
centroid_connector_file = cd + "centroid_connector_builder/centroid_links.csv"
node_coordinates_file = cd + "centroid_connector_builder/node_coordinates.csv"

In [307]:
#read input files
osm_edge_gdf = gpd.read_file(osm_edge_file)
osm_node_gdf = gpd.read_file(osm_node_file)
centroid_connector_df = pd.read_csv(centroid_connector_file)
node_coordinates_df = pd.read_csv(node_coordinates_file)

In [8]:
#define key param
zone_hi = 3030
external_hi = 3061

In [308]:
osm_node_gdf = osm_node_gdf.to_crs(epsg = 26915)
osm_node_gdf['x_reproj'] = osm_node_gdf['geometry'].apply(lambda p: p.x)
osm_node_gdf['y_reproj'] = osm_node_gdf['geometry'].apply(lambda p: p.y)

In [309]:
centroid_connector_df['c'] = centroid_connector_df[['A', 'B']].min(axis = 1)
centroid_connector_df['non_c'] = centroid_connector_df[['A', 'B']].max(axis = 1)

In [310]:
develop_new_cc_df = pd.merge(centroid_connector_df.groupby(['c', 'non_c']).count().reset_index().drop(['A','B'], axis = 1),
         centroid_connector_df.groupby(['c', 'non_c']).count().groupby('c').count()[['A']].rename(columns = {'A':'abm_num_load'}).reset_index(),
         how = 'left', 
         on = 'c')
develop_new_cc_df['osm_num_load'] = (4 / develop_new_cc_df['abm_num_load']).astype(int)
develop_new_cc_df = pd.merge(develop_new_cc_df, 
                            node_coordinates_df[['N', 'X', 'Y']],
                            how = 'left',
                            left_on = 'non_c',
                            right_on = 'N')

In [329]:
def find_new_load_point(abm_node_df, osm_node_gdf):
    inventory_node_ref = osm_node_gdf[['x_reproj', 'y_reproj']].values
    tree = cKDTree(inventory_node_ref)
    new_load_point_gdf = gpd.GeoDataFrame()
    for i in range(len(abm_node_df)):
        point = abm_node_df.iloc[i][['X', 'Y']].values
        n_neigh = abm_node_df.iloc[i]['osm_num_load']
        dd, ii = tree.query(point, k = n_neigh)
        if n_neigh == 1:
            add_gdf = gpd.GeoDataFrame(osm_node_gdf[['osmid', 'geometry']].iloc[ii]).transpose().reset_index(drop = True)
        else:
            add_gdf = gpd.GeoDataFrame(osm_node_gdf[['osmid', 'geometry']].iloc[ii]).reset_index(drop = True)
        add_gdf['c'] = int(abm_node_df.iloc[i]['c'])
        if i == 0:
            new_load_point_gdf = add_gdf.copy()
        else:
            new_load_point_gdf = new_load_point_gdf.append(add_gdf, ignore_index=True, sort=False)
        #print(i)
        #print(add_gdf)
    return new_load_point_gdf.rename(columns = {'geometry' : 'geometry_l'})

In [359]:
def generate_centroid_connectors(abm_load_ref_df, osm_node_gdf, abm_node_df):
    new_load_point_gdf = find_new_load_point(abm_load_ref_df, osm_node_gdf)
    new_load_point_gdf = pd.merge(new_load_point_gdf,
                                 abm_node_df[['N', 'X', 'Y']],
                                 how = 'left', 
                                 left_on = 'c',
                                 right_on = 'N')
    new_load_point_gdf['geometry_c'] = [Point(xy) for xy in zip(new_load_point_gdf['X'], new_load_point_gdf['Y'])]
    new_load_point_gdf.drop(['N', 'X', 'Y'], axis = 1, inplace = True)
    
    #centroid coordinates
    new_centroid_gdf = new_load_point_gdf.copy()[['c', 'geometry_c']]
    new_centroid_gdf.rename(columns = {'c' : 'osmid', 'geometry_c' : 'geometry'}, inplace = True)
    new_centroid_gdf.drop_duplicates(['osmid'], inplace = True)
    
    #inbound cc
    new_cc_in_gdf = new_load_point_gdf.copy()
    new_cc_in_gdf['geometry'] = [LineString(xy) for xy in zip(new_cc_in_gdf['geometry_l'], new_cc_in_gdf['geometry_c'])]
    new_cc_in_gdf = new_cc_in_gdf.rename(columns = {'osmid' : 'osmid_from', 'c' : 'osmid_to'})
    
    #outbound cc
    new_cc_out_gdf = new_load_point_gdf.copy()
    new_cc_out_gdf['geometry'] = [LineString(xy) for xy in zip(new_cc_out_gdf['geometry_c'], new_cc_out_gdf['geometry_l'])]
    new_cc_out_gdf = new_cc_out_gdf.rename(columns = {'osmid' : 'osmid_to', 'c' : 'osmid_from'})
    
    #bind
    new_cc_gdf = new_cc_in_gdf.append(new_cc_out_gdf, ignore_index = True, sort = False)
    new_cc_gdf.drop(['geometry_l', 'geometry_c'], axis = 1, inplace = True)
    
    #remove duplicates
    new_cc_gdf.drop_duplicates(['osmid_from', 'osmid_to'], inplace = True)
    
    return new_cc_gdf, new_centroid_gdf

In [360]:
new_cc_gdf, new_centroid_gdf = generate_centroid_connectors(develop_new_cc_df, osm_node_gdf, node_coordinates_df)

In [423]:
new_cc_gdf.crs = {'init' : 'epsg:26915'}
new_cc_gdf['LANES_CONS'] = 1
new_cc_gdf['CENTROID'] = int(1)
osm_edge_gdf = osm_edge_gdf.to_crs(epsg = 26915)
osm_edge_gdf['CENTROID'] = int(0)
osm_edge_gdf.rename(columns = {'U' : 'osmid_from', 'V' : 'osmid_to'}, inplace = True)
osm_edge_with_cc_gdf = osm_edge_gdf.append(new_cc_gdf, ignore_index = True, sort = False)
osm_edge_with_cc_gdf['ID'] = range(1, 1+len(osm_edge_with_cc_gdf))
osm_edge_with_cc_gdf.to_file(cd + "/centroid_connector_builder/osm_edges_with_cc.shp")

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [362]:
osm_node_with_centroid_gdf = osm_node_gdf.append(new_centroid_gdf, ignore_index = True, sort = False)
osm_node_with_centroid_gdf.crs = {'init' : 'epsg:26915'}
osm_node_with_centroid_gdf[['osmid', 'geometry']].to_file(cd + "/centroid_connector_builder/osm_nodes_with_centroid.shp")

In [424]:
pd.DataFrame(osm_node_with_centroid_gdf[['osmid', 'geometry']]).to_csv(cd + "/centroid_connector_builder/osm_nodes_with_centroid.csv",
                                                                      index = False)